In [5]:
import zombie_imp
import csv
import serial
import time 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import pywt

ModuleNotFoundError: No module named 'pywt'

In [2]:
# Initialize the Serial port
ser = serial.Serial('COM3', 115200)  # Change the port and baud rate according to transmitter

# create the CSV file
with open('sensor_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Cell 1', 'Cell 2', 'Cell 3'])  

    # read the data from port and store in CSV file
    try:
        while True:
            data = ser.readline().decode('utf-8').strip()  # read data from port
            if data:
                values = data.split(' ')  # split with space
                if len(values) == 3:  # make sure there are three values
                    writer.writerow([int(val) for val in values])  # store the data
    except KeyboardInterrupt:  # stop iteration by Ctrl+C
        pass

In [ ]:
# read the csv file
df = pd.read_csv('sensor_data.csv')

# get the data of each solar cell
solar_cell_1 = df['Cell 1']
solar_cell_2 = df['Cell 2']
solar_cell_3 = df['Cell 3']

# apply DWT for the current value of each solar cell
coeffs_1 = pywt.wavedec(solar_cell_1, 'haar', level=1)
coeffs_2 = pywt.wavedec(solar_cell_2, 'haar', level=1)
coeffs_3 = pywt.wavedec(solar_cell_3, 'haar', level=1)

print("Coefficients for solar_cell_1:", coeffs_1)
print("Coefficients for solar_cell_2:", coeffs_2)
print("Coefficients for solar_cell_3:", coeffs_3)